# Visualize the Composite Images

## Environment Set Up

In [ ]:
import ee
ee.Initialize()

In [ ]:
# Import Libraries
import geemap
from google.cloud import storage
import re

In [ ]:
# Set up access to abrupt_thaw
storage_client = storage.Client(project="AbruptThawMapping")
abrupt_thaw = storage_client.get_bucket('abrupt_thaw')

## Define Functions

In [ ]:
# Set Properties to allow filtering
def setID(image):
    img_id = image.id();
    img_prop = image.setMulti({'ID': img_id});
    img_prop = ee.Image(img_prop);
    return img_prop;

In [ ]:
# Mask nodata
def mask_0(image):
    mask = image.gt(0)
    return image.updateMask(mask)

## Import Data

In [ ]:
# Import Data
# Maxar
maxar = ee.Image("users/gfiske/FrostCraters/maxar_imagery_v2");

# Import Planet Data from GCS
planet = ee.ImageCollection('projects/abruptthawmapping/assets/yg_val_regions_composites')
planet = planet.map(setID)
print(planet.first().bandNames().getInfo())

# Polygons
polygons = ee.FeatureCollection('users/gfiske/FrostCraters/rts_polygons_for_Yili_May_2022')

In [ ]:
# Get Sentinel-2 Mosaic to Compare, as well
s2 = ee.ImageCollection('projects/abruptthawmapping/assets/yg_val_regions_sentinel_reference')
s2 = planet.map(setID)

In [ ]:
# Mask the data to values greater than 0 (which is how nodata gets imported into GEE by default)
planet = planet.map(mask_0)

In [ ]:
# Prep Map
Map = geemap.Map()
Map.centerObject(planet)

In [ ]:
# Separate out Years
planet_2017 = planet.filter(ee.Filter.stringContains('ID', ee.String('2017')))
planet_2018 = planet.filter(ee.Filter.stringContains('ID', ee.String('2018')))
planet_2019 = planet.filter(ee.Filter.stringContains('ID', ee.String('2019')))
planet_2020 = planet.filter(ee.Filter.stringContains('ID', ee.String('2020')))
planet_2021 = planet.filter(ee.Filter.stringContains('ID', ee.String('2021')))
planet_all = planet.filter(ee.Filter.stringContains('ID', ee.String('all')))

In [ ]:
# Set visualization parameters
vis_params = {
    'min': 0,'max': 1200,
    'bands': ['red', 'green', 'blue'],
    'gamma': 1
}

In [ ]:
# Add layers
# Add Maxar to all maps
Map.addLayer(maxar, {'min':8, 'max':192}, 'Maxar')

# Add Sentinel-2 composite
Map.addLayer(s2, {'bands':['red', 'green', 'blue'], 'min':0, 'max':1200}, 'S2 All Years')

# add Planet composites
Map.addLayer(planet_2017, vis_params, 'Planet 2017')
Map.addLayer(planet_2018, vis_params, 'Planet 2018')
Map.addLayer(planet_2019, vis_params, 'Planet 2019')
Map.addLayer(planet_2020, vis_params, 'Planet 2020')
Map.addLayer(planet_2021, vis_params, 'Planet 2021')
Map.addLayer(planet_all, vis_params, 'Planet 2017-2021')

# add RTS Polygons
Map.addLayer(polygons, {'color': 'red'}, 'RTS')

In [ ]:
# Display the map
Map